#### Literal Type

> ##### Note
>
> This is a new feature of the Python standard library as of Python 3.8; prior to Python 3.8, it requires the `typing-extensions` package.

_pydantic_ supports the use of `typing.Literal` (or `typing_extensions.Literal` prior to Python 3.8) as a lightweight way to specify that a field may accept only specific literal values.

In [1]:
from typing import ClassVar, List, Literal, Optional, Union
from pydantic import BaseModel, ValidationError

In [2]:
class Pie(BaseModel):
    flavor: Literal["apple", "pumpkin"]

In [3]:
Pie(flavor="apple")

Pie(flavor='apple')

In [4]:
Pie(flavor="pumpkin")

Pie(flavor='pumpkin')

In [5]:
try:
    m = Pie(flavor="cherry")
    print(m)
except ValidationError as e:
    print(str(e))

1 validation error for Pie
flavor
  unexpected value; permitted: 'apple', 'pumpkin' (type=value_error.const; given=cherry; permitted=('apple', 'pumpkin'))


One benefit of this field type is that it can be used to check for equality with one or more specific values without needing to declare custom validators.

In [6]:
class Cake(BaseModel):
    kind: Literal["cake"]
    required_utensils: ClassVar[List[str]] = ["fork", "knife"]

In [7]:
class IceCream(BaseModel):
    kind: Literal["icecream"]
    required_utensils: ClassVar[List[str]] = ["spoon"]

In [8]:
class Meal(BaseModel):
    dessert: Union[Cake, IceCream]

In [9]:
print(f"{type(Meal(dessert={'kind': 'cake'}).dessert).__name__ = }")

type(Meal(dessert={'kind': 'cake'}).dessert).__name__ = 'Cake'


In [10]:
print(f"{type(Meal(dessert={'kind': 'icecream'}).dessert).__name__ = }")

type(Meal(dessert={'kind': 'icecream'}).dessert).__name__ = 'IceCream'


In [11]:
try:
    m = Meal(dessert={"kind": "pie"})
    print(m)
except ValidationError as e:
    print(str(e))

2 validation errors for Meal
dessert -> kind
  unexpected value; permitted: 'cake' (type=value_error.const; given=pie; permitted=('cake',))
dessert -> kind
  unexpected value; permitted: 'icecream' (type=value_error.const; given=pie; permitted=('icecream',))


With proper ordering in an annotated `Union`, you can use this to parse types of decreasing specificity.

In [12]:
class Dessert(BaseModel):
    kind: str

In [13]:
class SuperPie(BaseModel):
    kind: Literal["pie"]
    flavor: Optional[str]

In [14]:
class ApplePie(Pie):
    flavor: Literal["apple"]

In [15]:
class PumpkinPie(Pie):
    flavor: Literal["pumpkin"]

In [16]:
class DeliciousMeal(BaseModel):
    dessert: Union[ApplePie, PumpkinPie, SuperPie, Dessert]

In [17]:
print(f"{type(DeliciousMeal(dessert={'kind': 'pie', 'flavor': 'apple'}).dessert).__name__ = }")

type(DeliciousMeal(dessert={'kind': 'pie', 'flavor': 'apple'}).dessert).__name__ = 'ApplePie'


In [18]:
print(f"{type(DeliciousMeal(dessert={'kind': 'pie', 'flavor': 'pumpkin'}).dessert).__name__ = }")

type(DeliciousMeal(dessert={'kind': 'pie', 'flavor': 'pumpkin'}).dessert).__name__ = 'PumpkinPie'


In [19]:
print(f"{type(DeliciousMeal(dessert={'kind': 'pie'}).dessert).__name__ = }")

type(DeliciousMeal(dessert={'kind': 'pie'}).dessert).__name__ = 'SuperPie'


In [20]:
print(f"{type(DeliciousMeal(dessert={'kind': 'cake'}).dessert).__name__ = }")

type(DeliciousMeal(dessert={'kind': 'cake'}).dessert).__name__ = 'Dessert'
